In [1]:
from ortools.linear_solver import pywraplp
import seaborn as sns
import numpy as np
import pandas as pd
import time
from models.problem_setup import setup

## Model Implementation Empirical Time Assessment

In this section, we proceed to study how different algorithms and solvers succeed in finding a solution to our problem with our model implementation.

In [ ]:
parameters = pywraplp.MPSolverParameters()

#### Solver/Optimizer Analysis

Despite not proving the solver in practical lessons, we include solvers like __Gurobi__, __XPRESS__ or __CPLEX__ in their suitable versions for MILP problems.

In [ ]:
solver_params = {}

solver_types = ["GLOP","SCIP","GUROBI","CPLEX","XPRESS"]
for algorithm in solver_types:
    solver = pywraplp.Solver.CreateSolver(algorithm)
    
    # setup function works as a wraper that takes a solver redefined with 
    # certain variations in parameters or even solver type in the previous line
    # together with the variables, constraints and objective function.
    # It will return the solver with the appropiate changes in the parameters provided
    
    solver = setup(INTRODUCE VARIABLES, CONSTRAINTS AND OBJECTIVE FUNCTION,solver)
    solver_params[algorithm] = []
    for i in range(30):
        t = time.time()
        solver.Solve()
        solver_params[algorithm].append(time.time()-t)



In [ ]:
sns.barplot()

In [6]:
pywraplp.MPSolverParameters()

<ortools.linear_solver.pywraplp.MPSolverParameters; proxy of <Swig Object of type 'operations_research::MPSolverParameters *' at 0x000001DFC15BE010> >